In [20]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import os
import glob
import time

# Use Hugging face (slow)

In [22]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

# # Load model and tokenizer
# # model_id = "LiquidAI/LFM2-1.2B" # <- or LFM2-700M or LFM2-350M
# # model_id = "LiquidAI/LFM2-700M"
# model_id = "LiquidAI/LFM2-350M-ENJP-MT"
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",
#     dtype="bfloat16",
# #   attn_implementation="flash_attention_2" <- uncomment on compatible GPU
# )
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# # streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# def fluid_translate(prompt, lang='jp2en'):
#     if lang == "jp2en":
#         system_prompt = "Translate to English"
#     elif lang == "en2jp":
#         system_prompt = "Translate to Japanese"
#     else:
#         raise Exception("lang not supported")

#     input_ids = tokenizer.apply_chat_template(
#         [{"role": "system", "content": system_prompt },
#         {"role": "user", "content": prompt}],
#         add_generation_prompt=True,
#         return_tensors="pt",
#         tokenize=True,
#     ).to(model.device)

#     output = model.generate(
#         input_ids,
#         do_sample=True,
#         temperature=0.1,
#         min_p=0.15,
#         repetition_penalty=1.05,
#         max_new_tokens=16000,
#         # streamer=streamer,
#     )
#     # Decode the output to get the text
#     full_text = tokenizer.decode(output[0], skip_special_tokens=True)

#     # Extract only the assistant's response
#     if "assistant" in full_text:
#         generated_text = full_text.split("assistant")[-1].strip()
#     else:
#         generated_text = full_text

#     return generated_text

# # test
# prompt = "What is C. elegans?"
# fluid_translate(prompt, lang='jp2en')

# Use llama.cpp (faster)

In [23]:
import os
import requests
from pathlib import Path
from llama_cpp import Llama
from contextlib import redirect_stderr
from transformers import AutoTokenizer


def download_model(model_name,model_family,model_guuf_file):
        with open(os.devnull, 'w') as devnull:
            with redirect_stderr(devnull):
                if not os.path.exists(model_guuf_file):
                    url = f"https://huggingface.co/LiquidAI/{model_family}/resolve/main/{model_name}"
                    download_gguf_model(url)

def download_gguf_model(model_url: str, models_folder: str = "../models") -> str:
    """
    Download a GGUF model file from a Hugging Face URL to the models folder.
    
    Args:
        model_url: The HTTPS URL to the GGUF model file
        models_folder: The destination folder (default: "../models")
    
    Returns:
        str: The local path to the downloaded model file
    """
    # Create models folder if it doesn't exist
    Path(models_folder).mkdir(parents=True, exist_ok=True)
    
    # Extract filename from URL
    filename = model_url.split("/")[-1].split("?")[0]
    local_path = os.path.join(models_folder, filename)
    
    # Check if file already exists
    if os.path.exists(local_path):
        print(f"[yellow]⚠️  Model already exists:[/yellow] [dim]{local_path}[/dim]")
        return local_path
    
    print(f"[cyan]⬇️  Downloading model from:[/cyan] [dim]{model_url}[/dim]")
    print(f"[cyan]📁 Saving to:[/cyan] [dim]{local_path}[/dim]")
    
    try:
        # Stream download with progress
        response = requests.get(model_url, stream=True)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        block_size = 8192
        downloaded = 0
        
        with open(local_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=block_size):
                if chunk:
                    f.write(chunk)
                    downloaded += len(chunk)
                    if total_size > 0:
                        progress = (downloaded / total_size) * 100
                        print(f"\r[cyan]Progress:[/cyan] {progress:.1f}% ({downloaded / (1024*1024):.1f} MB / {total_size / (1024*1024):.1f} MB)", end="")
        
        print(f"\n[green]✅ Download complete![/green]")
        return local_path
        
    except requests.exceptions.RequestException as e:
        print(f"\n[red]❌ Download failed: {e}[/red]")
        # Clean up partial download
        if os.path.exists(local_path):
            os.remove(local_path)
        raise
    
# Suppress llama.cpp verbose output
def load_llm(model_path, n_ctx=8000):
    """
    Load a llama.cpp model with specified context window size.
    
    Args:
        model_path: Path to the GGUF model file
        n_ctx: Maximum context window size in tokens (default: 2048)
               Common values: 512, 1024, 2048, 4096, 8192, 16384, 32768
    """
    with open(os.devnull, 'w') as devnull:
        with redirect_stderr(devnull):
            llm = Llama(
                model_path=model_path,
                n_ctx=n_ctx,
                verbose=False,
                temperature=0.1,
            )
            return llm


# Extract PII double step: General (1.2B) + Nano (1.2 EXT)

In [24]:
SYSTEM_PROMPT = "You're are helpful assistant. Be concise and accurate."

MODEL_NAME="LFM2-1.2B-Q8_0.gguf"
MODEL_GGUF_FILE = f"../models/{MODEL_NAME}"
MODEL_FAMILY="LFM2-1.2B-GGUF"

def get_user_prompt_pii(contract):
    return f"""
Below is a text from a Japanese Legal contract. 
It contains Personal Identifiable information in Japanese.
You are an expert in Japanese Personal Identifiable Information Detection.
Your task it to 
1. carefully read the content identify Personal Information
2. extract four type of personal identifiable information: such as person names, company names, addresses, and phone numbers"

---- Japanese Legal contract ----
{contract}
"""

def find_pii(fn):
    """
    Translate a Japanese text file to English and save as fn_en.txt
    
    Args:
        fn: Path to the input text file
    """

    with open(fn, 'r', encoding='utf-8') as f:
        contract = f.read()
    history = [{"role": "system", "content": SYSTEM_PROMPT}]
    history.append({"role": "user", "content": get_user_prompt_pii(contract)})
    
    model_id = "LiquidAI/LFM2-350M-ENJP-MT"
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    tokens = tokenizer.encode(str(history))
    num_tokens = len(tokens)
    _ = print(f"Prompt has {num_tokens} tokens")
    
    # Translate using fluid_translate
    t0=time.time()
    # Increase context window to handle larger contracts
    llm_pii = load_llm(MODEL_GGUF_FILE, n_ctx=16000)
    output = llm_pii.create_chat_completion(messages=history)
    unstructured_pii = output["choices"][0]["message"]["content"]

    dt =time.time()-t0
    _ = print(f"took {dt} to extraxt pii {num_tokens} ({num_tokens/dt:.2f} token/sec)")
    return unstructured_pii

unstructured_pii_folder = '../data/unstructured_pii/'

# Create output folder if it doesn't exist
os.makedirs(unstructured_pii_folder, exist_ok=True)

txt_folder='../data/txt/'
fns = glob.glob(f'{txt_folder}*.txt')
fns = sorted([fn.split('/')[-1] for fn in fns if 'tesse' in fn ])

for fn in fns:
    output_path = os.path.join(unstructured_pii_folder, fn)
    print(output_path)
    if not os.path.exists(output_path):
        unstructured_pii=find_pii(txt_folder+fn)
        
        # Save unstructured_pii text to unstructured_pii_folder/fn
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(unstructured_pii)
        print(f"Saved unstructured PII to: {output_path}")


../data/unstructured_pii/tesseract_Redactor_sample_0001.txt
../data/unstructured_pii/tesseract_Redactor_sample_0002.txt
../data/unstructured_pii/tesseract_Redactor_sample_0003.txt
../data/unstructured_pii/tesseract_Redactor_sample_0004.txt
../data/unstructured_pii/tesseract_Redactor_sample_0005.txt
../data/unstructured_pii/tesseract_Redactor_sample_0006.txt
../data/unstructured_pii/tesseract_Redactor_sample_0007.txt
../data/unstructured_pii/tesseract_Redactor_sample_0008.txt
../data/unstructured_pii/tesseract_Redactor_sample_0009.txt
../data/unstructured_pii/tesseract_Redactor_sample_0010.txt
../data/unstructured_pii/tesseract_Redactor_sample_0011.txt
../data/unstructured_pii/tesseract_Redactor_sample_0012.txt
../data/unstructured_pii/tesseract_Redactor_sample_0013.txt
../data/unstructured_pii/tesseract_Redactor_sample_0014.txt
../data/unstructured_pii/tesseract_Redactor_sample_0015.txt
../data/unstructured_pii/tesseract_Redactor_sample_0016.txt
../data/unstructured_pii/tesseract_Redac

In [ ]:
MODEL_GGUF_FILE_EXTRACT = "../models/LFM2-1.2B-Extract-F16.gguf"
def get_user_prompt_extract(txt):
    return f"""
-----
{txt}
"""

SYSTEM_PROMPT = """Identify and extract information matching the following schema. No other information has to be extracted. Adhere to the 4 fields schema, and to the exact 4 english key names

"full_name": name of the person" (string),
"company_name": name of the company (string) ,
"address": address of the company (string) ,
"phone_number": phone number (string)

"""


unstructured_pii_folder = '../data/unstructured_pii/'
json_pii = '../data/json_pii/'

os.makedirs(json_pii, exist_ok=True)

txt_folder='../data/txt/'
fns = glob.glob(f'{unstructured_pii_folder}*.txt')
fns = sorted([fn.split('/')[-1] for fn in fns if 'padd' in fn ])

for fn in fns:
    output_path = os.path.join(json_pii, fn).replace('.txt','.json')
    print(output_path)
    if True or not os.path.exists(output_path):
        with open(unstructured_pii_folder+fn, 'r', encoding='utf-8') as f:
            unstructured_pii = f.read()
        history = [{"role": "system", "content": SYSTEM_PROMPT}]
        history.append({"role": "user", "content": get_user_prompt_extract(unstructured_pii)})
        llm_extract = load_llm(MODEL_GGUF_FILE_EXTRACT)

        output = llm_extract.create_chat_completion(messages=history)
        pii = output["choices"][0]["message"]["content"]

        # Save unstructured_pii text to unstructured_pii_folder/fn

        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(pii)
        print(f"Saved json_pii to: {output_path}")




../data/json_pii/tesseract_Redactor_sample_0001.json


165

Saved json_pii to: ../data/json_pii/tesseract_Redactor_sample_0001.json
../data/json_pii/tesseract_Redactor_sample_0002.json


148

Saved json_pii to: ../data/json_pii/tesseract_Redactor_sample_0002.json
../data/json_pii/tesseract_Redactor_sample_0003.json


...

Saved json_pii to: ../data/json_pii/tesseract_Redactor_sample_0034.json
../data/json_pii/tesseract_Redactor_sample_0035.json


225

Saved json_pii to: ../data/json_pii/tesseract_Redactor_sample_0035.json
../data/json_pii/tesseract_Redactor_sample_0036.json


142

Saved json_pii to: ../data/json_pii/tesseract_Redactor_sample_0036.json
../data/json_pii/tesseract_Redactor_sample_0037.json


# Translate

In [ ]:
# https://huggingface.co/LiquidAI/LFM2-350M-ENJP-MT-GGUF/resolve/main/LFM2-350M-ENJP-MT-F32.gguf?download=true

MODEL_NAME="LFM2-350M-ENJP-MT-Q4_0.gguf"
MODEL_GGUF_FILE = f"../models/{MODEL_NAME}"
MODEL_FAMILY="LFM2-350M-ENJP-MT-GGUF"

def fluid_translate(prompt, lang='jp2en'):
    if lang == "jp2en":
        system_prompt = "Translate to English"
    elif lang == "en2jp":
        system_prompt = "Translate to Japanese"
    else:
        raise Exception("lang not supported")
    download_model(MODEL_NAME,MODEL_FAMILY,MODEL_GGUF_FILE)
    llm = load_llm(MODEL_GGUF_FILE)
    history = [{"role": "system", "content": system_prompt}]
    history.append({"role": "user", "content": prompt})
    output = llm.create_chat_completion(messages=history)
    generated_message = output["choices"][0]["message"]["content"]
    return generated_message

In [ ]:
def jp2en(fn, override=False):
    """
    Translate a Japanese text file to English and save as fn_en.txt
    
    Args:
        fn: Path to the input text file
    """
    base_name = os.path.splitext(fn)[0]
    output_fn = f"{base_name}_en.txt"
    if not os.path.exists(output_fn) or override:     
        # Read the content of the file
        with open(fn, 'r', encoding='utf-8') as f:
            prompt = f.read()
        
        model_id = "LiquidAI/LFM2-350M-ENJP-MT"
        tokenizer = AutoTokenizer.from_pretrained(model_id)

        # Print how many tokens is the prompt
        tokens = tokenizer.encode(prompt)
        num_tokens = len(tokens)
        print(f"Prompt has {num_tokens} tokens")
        
        # Translate using fluid_translate
        t0=time.time()
        translated_text = fluid_translate(prompt, lang='jp2en')
        dt =time.time()-t0
        print(f"took {dt} to translate {num_tokens} ({num_tokens/dt:.2f} token/sec)")
        # Write translated text to file
        with open(output_fn, 'w', encoding='utf-8') as f:
            f.write(translated_text)
        
        print(f"Translated {fn} -> {output_fn}")
        return output_fn


fns = glob.glob('../data/txt/*.txt')
fns = [fn for fn in fns if "_en" not in fn]
fns
for fn in fns:
    jp2en(fn, True)